# Day 12: Passage Pathing

https://adventofcode.com/2021/day/12

## Part 1

With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just a path - the only way to know if you've found the best path is to find all of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:
```
start-A
start-b
A-c
A-b
b-d
A-end
b-end
```
This is a list of how all of the caves are connected. You start in the cave named start, and your destination is the cave named end. An entry like b-d means that cave b is connected to cave d - that is, you can move between them.

So, the above cave system looks roughly like this:
```
    start
    /   \
c--A-----b--d
    \   /
     end
```
Your goal is to find the number of distinct paths that start at start, end at end, and don't visit small caves more than once. There are two types of caves: big caves (written in uppercase, like A) and small caves (written in lowercase, like b). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should visit small caves at most once, and can visit big caves any number of times.

Given these rules, there are 10 paths through this example cave system:
```
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end
```
(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave d is never visited by any path: to do so, cave b would need to be visited twice (once on the way to cave d and a second time when returning from cave d), and since cave b is small, this is not allowed.

Here is a slightly larger example:
```
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
```
The 19 paths through it are as follows:
```
start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end
```
Finally, this even larger example has 226 paths through it:
```
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
```

**How many paths through this cave system are there that visit small caves at most once?**

In [1]:
from IPython.display import Markdown
import copy

In [2]:
#infile = 'test-input1.txt'
#infile = 'test-input2.txt'
#infile = 'test-input3.txt'
infile = 'input.txt'

# Represent connection graph as dict with
# each node as a key and all available neighbors
# in a set as the value
cavegraph = dict()
with open(infile, 'r') as fid:
    for line in fid:
        node, neigh = line.strip().split('-')
        # Never include 'start' as a neighbor since we
        # can never go back there
        if node in cavegraph:
            if 'start' != neigh:
                cavegraph[node].add(neigh)
        else:
            if 'start' != neigh:
                cavegraph[node] = {neigh}
        if neigh in cavegraph:
            if 'start' != node:
                cavegraph[neigh].add(node)
        else:
            if 'start' != node:
                cavegraph[neigh] = {node}

In [3]:
#cavegraph

In [4]:
def pr(val):
    print(val, end=',')

In [5]:
def pathway(node, seen=set(), thisroute=list(), routes=list()):
    # Recursive function to traverse all valid cave paths
    # allowing for visiting each small cave at most once
    #print("=={}==".format(node))
    thisroute.append(node)
    #print("This route: {}".format(thisroute))
    if 'end' == node:
        routes.append(thisroute)
        #print('...Hit the end: {}'.format(routes))
        return routes
    if node in seen:
        # Already saw this small cave, not a valid route
        return routes
    if node == node.lower():
        # Small cave, add to seen
        seen.add(node)
    # From this node, try every neighbor
    #print("\n... Available neighbors: {}".format(cavegraph[node]))
    #print("... Seen: {}".format(seen))
    # Cache thisroute and seen before descent
    thisroute0 = thisroute.copy()
    seen0 = seen.copy()
    for neigh in cavegraph[node]:
        # Restore cached thisroute and seen for this descent
        thisroute1 = thisroute0.copy()
        seen1 = seen0.copy()
        #pr('({})'.format(neigh))
        #print('__{}__'.format(neigh))
        #print('thisroute1: {}'.format(thisroute1))
        routes = pathway(neigh, seen=seen1, thisroute=thisroute1, routes=routes)
    return routes

In [6]:
routes = pathway('start')
#print()
#print('-='*20)
#for r in routes:
#    print(','.join(r))

In [7]:
npaths = len(routes)
Markdown("The number of paths is **{}**".format(npaths))

The number of paths is **5104**

## Part Two

After reviewing the available paths, you realize you might have time to visit a single small cave twice. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named start and end can only be visited exactly once each: once you leave the start cave, you may not return to it, and once you reach the end cave, the path must end immediately.

Now, the 36 possible paths through the first example above are:
```
start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
```
The slightly larger example above now has 103 paths through it, and the even larger example now has 3509 paths through it.

Given these new rules, **how many paths through this cave system are there?**

In [8]:
known_paths = set('''start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
'''.split())
#known_paths

In [9]:
def rls(route):
    # Return route list as csv string
    return ','.join(route)

In [10]:
#watchstr = 'start,A,b,A,b'
#watchstr = 'start,A,b,A,c,A,b,A,c'
#watchstr = 'start,A,b,A,b,A,c,A,c'
#watchstr = 'start,A,b,A'
watchstr = 'start,A,c,A,b,A,c,A'
def pathway2(node, seen=set(), seenbuffer=list(), smalldone=False,
             thisroute=list(), routes=list()):
    # Recursive function to traverse all valid cave paths
    # allowing for visiting a single small cave twice but all
    # other small caves at most once
    ###print("=={}==".format(node))
    thisroute.append(node)
    ###if rls(thisroute).startswith(watchstr):
    ###    print("*"*20)
    ###    print("WATCH... node: {}; seenbuffer: {}".format(node, seenbuffer))
    ###print("This route: {}; seen: {}; seenbuffer: {}, smalldone: {}".format(
    ###    rls(thisroute), rls(seen), rls(seenbuffer), smalldone))
    #print("WATCH... node: {}; seenbuffer: {}".format(node, seenbuffer))
    ###print("... Available neighbors: {}".format(cavegraph[node]))
    if 'end' == node:
        routes.append(thisroute)
        ###print('...HIT THE END')
        ###print('-'*40)
        return routes
    if node in seen:
        # Already saw this small cave, not a valid route
        ###print('... EXITING DUE TO {} in seen: {}'.format(node, rls(seen)))
        return routes
    #if node in seenbuffer and smalldone:
        # Already saw this small cave once and visited another
        # small cave twice... not a valid route
    #    return routes
    if node == node.lower():
        ###print('lower node: {}'.format(node))
        if smalldone:
            seen = seen.union(set(seenbuffer))
            seen.add(node)
        # If this is the first time we've seen a small cave, we might
        # come back to visit it a second time
        #if node not in seenbuffer and node not in {'start', 'end'}:
        #print("Small cave: {}".format(node))
        #if 0 == len(seenbuffer) and node not in {'start', 'end'}:
        elif node not in {'start', 'end'} and node not in seenbuffer:
            ###print("Buffering: {}".format(node))
            #seenbuffer.add(node)
            seenbuffer.append(node)
        #elif smalldone:
        #    # If smalldone is True, this must go in seen
        #    #seen.add(node)
        #    # Add everything in seenbuffer to seen
        #    seen = seen.union(set(seenbuffer))
        else:
            # Node WAS in the seenbuffer but was not in seen (but may be start/end)
            # If we've already visited this node before, flag that
            # we may no longer visit small caves after this
            ###print("smalldone: {}; node: {}; seen: {}; seenbuffer: {}".format(
            ###        smalldone, node, rls(seen), rls(seenbuffer)))
            #if smalldone:
            #    # smalldone was already set True (and this node was in
            #    # seenbuffer)
            #    print('ADDING {} to seen because smalldone was True and this node was in screenbuffer'.
            #         format(node))
            #    #seen.add(node)
            #    # Add everything in seenbuffer to seen
            #    seen = seen.union(set(seenbuffer))
            #else:
            # smalldone was False
            # Visiting small caves is herein forbidden
            if node not in {'start', 'end'}:
                smalldone = True
                ###print("Setting smalldone to TRUE")
                seen = seen.union(set(seenbuffer))
                
    # From this node, try every neighbor
    #print("... Available neighbors: {}".format(cavegraph[node]))
    #print("... Seen: {}".format(seen))
    # Cache this route and seen before descent
    thisroute0 = thisroute.copy()
    seen0 = seen.copy()
    seenbuffer0 = seenbuffer.copy()
    smalldone0 = copy.copy(smalldone)
    for neigh in cavegraph[node]:
        # Restore cached thisroute and seen for this descent
        thisroute1 = thisroute0.copy()
        seen1 = seen0.copy()
        seenbuffer1 = seenbuffer0.copy()
        smalldone1 = copy.copy(smalldone0)
        #pr('({})'.format(neigh))
        #print('__{}__'.format(neigh))
        #print('thisroute1: {}'.format(thisroute1))
        routes = pathway2(neigh, seen=seen1, seenbuffer=seenbuffer1,
                          smalldone=smalldone1,
                          thisroute=thisroute1, routes=routes)
    return routes

routes2 = pathway2('start')
routes2a = {','.join(x) for x in routes2}
npaths2 = len(routes2)
#display(Markdown("The number of paths is **{}**".format(npaths2)))

In [11]:
# TESTING
# Missed routes
#miss_routes = known_paths - routes2a
#print('=-'*20)
#print("MISSED ROUTES:")
#for x in miss_routes:
#    print(x)

# Invalid routes
#invalid_routes = routes2a - known_paths
#print('=-'*20)
#print("INVALID ROUTES:")
#for x in invalid_routes:
#    print(x)

#print('=-'*20)
#print("FOUND ROUTES:")
#for r in routes2:
#    print(','.join(r))    